# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.6")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.6")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpnxaapqu1/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.6
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift
[3/4] Compiling SwiftCoreMLTools Activations.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalled

In [2]:
import Foundation
import SwiftCoreMLTools
import TensorFlow
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"./data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

var index = Set<Int>()

while index.count < numRecords {
    index.insert(Int.random(in: 0..<numRecords))
}

let randomDataRecords = index.map{ dataRecords[$0] }

let dataFeatures = randomDataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = randomDataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Categorize Categorical Features with Ordinal values

In [6]:
var categoricalValues = Array(repeating: Set<Int32>(), count: 2)

for record in categoricalFeatures {
    categoricalValues[0].insert(record[0])
    categoricalValues[1].insert(record[1])
}

let sortedCategoricalValues = [categoricalValues[0].sorted(), categoricalValues[1].sorted()]

let ordinalCategoricalFeatures = categoricalFeatures.map{ [Int32(sortedCategoricalValues[0].firstIndex(of:$0[0])!), 
                                                           Int32(sortedCategoricalValues[1].firstIndex(of:$0[1])!)] }

## Split Train and Test

In [7]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(ordinalCategoricalFeatures[0..<numTrainRecords]))
let xCategoricalAllTest = matrixTranspose(Array(ordinalCategoricalFeatures[numTrainRecords...]))
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, 1]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, 1]))
}

let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [8]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[3.7090604, 12.055555, 11.054204, 0.5516017,   6.27021, 67.946434, 3.8624096,  405.0469,
  18.481758, 356.45288, 12.810893]] [[  9.113623,  24.196712,  6.8583913, 0.11535243, 0.70409226,  28.822058,   2.176942,  166.97446,
   2.1447299,   92.18375,  7.3877473]]


In [9]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [10]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 1] [405, 1] [405, 1]
Testing shapes  [101, 11] [101, 1] [101, 1] [101, 1]


# Model

In [11]:
struct MultiInputs<N: Differentiable, C>: Differentiable {
  var numerical: N
  
  @noDerivative
  var categorical: C

  @differentiable
  init(numerical: N, categorical: C) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Module {
    var embedding1 = TensorFlow.Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = TensorFlow.Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
    var allInputConcatLayer = Dense<Float>(inputSize: (11 + 2 + 5), outputSize: 64, activation: relu)
    var hiddenLayer = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
        let embeddingOutput1 = embedding1(input.categorical[0])
        let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
            TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[2]]))
        let embeddingOutput2 = embedding2(input.categorical[1])
        let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
            TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[2]]))
        let allConcat = Tensor<Float>(concatenating: [input.numerical[0], embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
        return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
    }
}

var model = RegressionModel()

# Training

In [12]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [13]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [14]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                         categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                       XCategoricalTrain[1][batchStart..<batchEnd]])
            let logits = model(multiInput)
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                     categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                   XCategoricalTrain[1][batchStart..<batchEnd]])
        let logits = model(multiInput)
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 529.0884, MAE: 21.016209
Epoch 2: MSE: 449.94937, MAE: 19.12399
Epoch 3: MSE: 365.3185, MAE: 16.928745
Epoch 4: MSE: 275.31314, MAE: 14.302202
Epoch 5: MSE: 190.93709, MAE: 11.408555
Epoch 6: MSE: 123.30173, MAE: 8.57655
Epoch 7: MSE: 79.491295, MAE: 6.537755
Epoch 8: MSE: 55.950478, MAE: 5.202893
Epoch 9: MSE: 44.580727, MAE: 4.5634108
Epoch 10: MSE: 37.279842, MAE: 4.168285
Epoch 11: MSE: 32.211872, MAE: 3.87214
Epoch 12: MSE: 28.353092, MAE: 3.6448083
Epoch 13: MSE: 25.91916, MAE: 3.442046
Epoch 14: MSE: 23.809538, MAE: 3.289499
Epoch 15: MSE: 22.509584, MAE: 3.1895945
Epoch 16: MSE: 21.451454, MAE: 3.1035998
Epoch 17: MSE: 20.647305, MAE: 3.0469809
Epoch 18: MSE: 19.880947, MAE: 2.9606042
Epoch 19: MSE: 19.185884, MAE: 2.9118884
Epoch 20: MSE: 18.542118, MAE: 2.8709092
Epoch 21: MSE: 17.781507, MAE: 2.7943006
Epoch 22: MSE: 17.459024, MAE: 2.762055
Epoch 23: MSE: 16.846321, MAE: 2.7299106
Epoch 24: MSE: 16.437881, MAE: 2.6759505
Epoch 25: MSE: 15.83022, MAE: 2.630382


Epoch 202: MSE: 4.773335, MAE: 1.4187578
Epoch 203: MSE: 4.700536, MAE: 1.4117839
Epoch 204: MSE: 4.821341, MAE: 1.4132253
Epoch 205: MSE: 4.633688, MAE: 1.4077339
Epoch 206: MSE: 4.5749235, MAE: 1.4185086
Epoch 207: MSE: 4.7668104, MAE: 1.4001565
Epoch 208: MSE: 4.6548786, MAE: 1.3993344
Epoch 209: MSE: 4.607834, MAE: 1.4057206
Epoch 210: MSE: 4.7243176, MAE: 1.402136
Epoch 211: MSE: 4.5674067, MAE: 1.4045955
Epoch 212: MSE: 4.3653684, MAE: 1.3834531
Epoch 213: MSE: 4.518463, MAE: 1.3781703
Epoch 214: MSE: 4.5822277, MAE: 1.3813382
Epoch 215: MSE: 4.346852, MAE: 1.3678825
Epoch 216: MSE: 4.326457, MAE: 1.3724117
Epoch 217: MSE: 4.6579447, MAE: 1.3765104
Epoch 218: MSE: 4.439329, MAE: 1.3778381
Epoch 219: MSE: 4.326722, MAE: 1.3732371
Epoch 220: MSE: 4.5309215, MAE: 1.3793356
Epoch 221: MSE: 4.4751353, MAE: 1.371872
Epoch 222: MSE: 4.3121037, MAE: 1.3609478
Epoch 223: MSE: 4.2852054, MAE: 1.3417854
Epoch 224: MSE: 4.308532, MAE: 1.3477483
Epoch 225: MSE: 4.2976003, MAE: 1.3700719
Epoch

Epoch 401: MSE: 2.3579879, MAE: 0.96951634
Epoch 402: MSE: 2.3830657, MAE: 0.964865
Epoch 403: MSE: 2.4054065, MAE: 0.9535614
Epoch 404: MSE: 2.505754, MAE: 0.9732986
Epoch 405: MSE: 2.350547, MAE: 0.95539606
Epoch 406: MSE: 2.2730005, MAE: 0.9564965
Epoch 407: MSE: 2.4066305, MAE: 0.9482577
Epoch 408: MSE: 2.4342864, MAE: 0.9730834
Epoch 409: MSE: 2.3175821, MAE: 0.9546349
Epoch 410: MSE: 2.2335894, MAE: 0.9459655
Epoch 411: MSE: 2.2834647, MAE: 0.9526716
Epoch 412: MSE: 2.345399, MAE: 0.94189507
Epoch 413: MSE: 2.3000488, MAE: 0.96481264
Epoch 414: MSE: 2.2842436, MAE: 0.9374243
Epoch 415: MSE: 2.3539307, MAE: 0.9442634
Epoch 416: MSE: 2.3343625, MAE: 0.9295705
Epoch 417: MSE: 2.3314943, MAE: 0.9519411
Epoch 418: MSE: 2.275968, MAE: 0.9440047
Epoch 419: MSE: 2.2362616, MAE: 0.92421794
Epoch 420: MSE: 2.1771753, MAE: 0.9354896
Epoch 421: MSE: 2.2149634, MAE: 0.93411595
Epoch 422: MSE: 2.148635, MAE: 0.93064976
Epoch 423: MSE: 2.168746, MAE: 0.9246274
Epoch 424: MSE: 2.3473446, MAE: 0.

# Test

In [15]:
Context.local.learningPhase = .inference

let multiInputTest = MultiInputs(numerical: [XNumericalTest],
                                 categorical: [XCategoricalTest[0],
                                               XCategoricalTest[1]])

let prediction = model(multiInputTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.10507653, MAE: 0.02263178


# Export

In [16]:
print(model.embedding1.embeddings.shape, model.embedding2.embeddings.shape)

[2, 2] [9, 5]


In [17]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.6"]) {
    Input(name: "numericalInput", shape: [11])
    Input(name: "categoricalInput1", shape: [1])
    Input(name: "categoricalInput2", shape: [1])
    Output(name: "output", shape: [1])
    NeuralNetwork {
        Embedding(name: "embedding1",
                     input: ["categoricalInput1"],
                     output: ["outEmbedding1"],
                     weight: model.embedding1.embeddings.transposed().flattened().scalars,
                     inputDim: 2,
                     outputChannels: 2)
        Permute(name: "permute1",
                     input: ["outEmbedding1"],
                     output: ["outPermute1"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten1",
                     input: ["outPermute1"],
                     output: ["outFlatten1"],
                     mode: .last)
        Embedding(name: "embedding2",
                     input: ["categoricalInput2"],
                     output: ["outEmbedding2"],
                     weight: model.embedding2.embeddings.transposed().flattened().scalars,
                     inputDim: 9,
                     outputChannels: 5)
        Permute(name: "permute2",
                     input: ["outEmbedding2"],
                     output: ["outPermute2"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten2",
                     input: ["outPermute2"],
                     output: ["outFlatten2"],
                     mode: .last)
        Concat(name: "concat",
                     input: ["numericalInput", "outFlatten1", "outFlatten2"],
                     output: ["outConcat"])
        InnerProduct(name: "dense1",
                     input: ["outConcat"],
                     output: ["outDense1"],
                     weight: model.allInputConcatLayer.weight.transposed().flattened().scalars,
                     bias: model.allInputConcatLayer.bias.flattened().scalars,
                     inputChannels: 11 + 2 + 5,
                     outputChannels: 64)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weight: model.hiddenLayer.weight.transposed().flattened().scalars,
                     bias: model.hiddenLayer.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weight: model.outputLayer.weight.transposed().flattened().scalars,
                     bias: model.outputLayer.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1)
    }
}

In [18]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "./s4tf_house_simplified_trained_model.mlmodel"))

In [19]:
print(XNumericalTest[0], XCategoricalTest[0][0], XCategoricalTest[1][0], "=>", YTest[0])
print(XNumericalTest[17], XCategoricalTest[0][17], XCategoricalTest[1][17], "=>", YTest[17])
print(XNumericalTest[87], XCategoricalTest[0][87], XCategoricalTest[1][87], "=>", YTest[87])

[  2.012596, -0.4982311,  1.0273249,  1.6332408, -0.6422595,  0.8484324, -0.9169787,  1.5628325,
 0.80114645, 0.37964535,  1.2587203] [0] [8] => [10.5]
[ -0.3916621,  -0.4982311,  -0.3636718, -0.27395806, -0.14658587,  0.76516277,  -0.6621258,
 -0.12604865,   1.1275274,  0.39309666, -0.06509334] [0] [4] => [20.1]
[-0.40257868,    2.808003,  -1.3901517,  -1.2795717,   1.4441153,   -1.174324,   1.5832256,
 -0.45544034,  -2.7424235,   0.4387662,  -1.1818073] [0] [1] => [33.3]
